<div style="max-width:800px; max-height:400px; margin-left: 0; margin-right: auto;">
<img src="future-web-long.jpg" alt="Drawing" />
</div>

# Portfolio Calculator

This notebook provides a simple program to help make informed decisions about buying into a fund or ETF based on some simplified assumptions. Such as:

* the growth rate of the asset 
* expense ratio (how much the fund/asset charges you to hold it)
* periodic contributions you make (eg. how much are you dollar cost averaging in)

This program in this notebook is divided into sections to help walk the user through each section. The first code block is just loading the libraries needed to run the program.

## Set the input variables

set the variables that will be used in the simulations

In [1]:
import pandas as pd 
import matplotlib.pyplot as plt 

principal = float(input("What is the amount of money you want to invest? (example: 100\n"))
growth_rate = input("what is the growth rate of the fund/ETF? (example: 10 --> 10% annual growth)")
growth_rate = float(growth_rate)/100.0
years = int(input("how long do you want to run the simulation in years? (example: 5)"))

print("\nloaded values:")
print("   initial amount: " + str(principal))
print("   growth rate:    " + str(growth_rate))
print("   years:          " + str(years))


What is the amount of money you want to invest?
 100
what is the growth rate of the fund/ETF? (eg 10 --> 10% annual growth) 10
how long do you want to run the simulation in years? 5



loaded values:
   initial amount: 100.0
   growth rate:    0.1
   years:          5


## Iteration Simulation

The below code will use simplified math (eg no exponents) and a for loop
to get some more fine grained information about what happens each year
to the fund.

Note the iteration simulation has additional parameters like
* 3 different simulations with different growth rates assumptions
    * conservative
    * middle of the road
    * aggressive
* expense ratios to simulate ETF or managed fund expenses which take away from growth
* yearly contributions

In [12]:
def print_series(name, series):
    """
        this function is just used to display the data
        in a tabular format
    """

    print("Portfolio: " + name)
    print("year | portfolio val")
    print("-----+--------------")
    cnt = 0
    divisor = "    | "
    for point in series:
        print(str(cnt) + divisor + str(point))
        cnt = cnt + 1
        if cnt >= 10:
            divisor = "   | "
            
            
def run_simulation_iterative(p, 
                             r,
                             years,
                             expense_ratio=0,
                             annual_contribution=0):
    """
        This section initiliazes the different variables
        There are 3 different series
        
        c - conservative
        m - middle road
        a - aggressive
        
        each of the variables in this section are intended to be tweaked
        for the specific scenario in consideration
    """
    
    # these are the growth rates (r -> rate)
    r_conservative = r # 0.05
    r_middle_road = r   
    r_aggressive = 0.15

    # compounded annually. See the for loop for details
        
    # This is assuming your contributions grow based on annual raises in income
    # to account for this raises are considered as a percentage 0.0 - 1 (0 - 100%)
    yr_raise = 0.00 
    
    # this is the expense ration of the fund/ETF/asset (0.01 -> 1% expense ratio)
    er_c = 0.0003
    er_m = expense_ratio
    er_a = 0.0

    # this is different values for the yearly contributions to the fund
    yr_contrib_c = 0
    yr_contrib_m = annual_contribution
    yr_contrib_a = 0
    
    # initialize the starting values for the different simulations
    port_worth_c = p
    port_worth_m = p
    port_worth_m_no_fee = p
    port_worth_a = p

    # initialize arrays to hold the data
    series_c = []
    series_m = []
    series_m_no_fee = []
    series_a = []

    # add the initial value of in the asset/fund to the array
    series_c.append(port_worth_c)
    series_m.append(port_worth_m)
    series_m_no_fee.append(port_worth_m_no_fee)
    series_a.append(port_worth_a)
    
    
    """
        this code loop assumes compound rate (n=1) is equal to one
        
        The algorithm is:
        
            portfolio worth = current-value + yearly-growth + your-contributions
            portfolio worth = new-value - the-expense-ratio
            
            then add the value for that year to the series
    """
    for i in range(0,years):
        port_worth_c = port_worth_c + (port_worth_c * r_conservative) + yr_contrib_c
        port_worth_c = port_worth_c * (1 - er_c)
        series_c.append(port_worth_c)

        port_worth_m = port_worth_m + (port_worth_m * r_middle_road) + yr_contrib_m
        port_worth_m = port_worth_m * (1 - er_m)
        series_m.append(port_worth_m)

        port_worth_m_no_fee = port_worth_m_no_fee + port_worth_m_no_fee * r_middle_road
        port_worth_m_no_fee = port_worth_m_no_fee 
        series_m_no_fee.append(port_worth_m_no_fee)

        port_worth_a = port_worth_a + (port_worth_a * r_aggressive) + yr_contrib_a
        port_worth_a = port_worth_a * (1 - er_a)
        series_a.append(port_worth_a)
        
        yr_contrib_c = yr_contrib_c + yr_contrib_c*yr_raise
        yr_contrib_m = yr_contrib_m + yr_contrib_c*yr_raise
        yr_contrib_a = yr_contrib_a + yr_contrib_c*yr_raise

        
    """
        Display all the final values of the different projections
    """
    print("")
    print("Value of the portfolios after " + str(years) + " years:")
    print("------------------------------------")    
    print("  conservative: $" + '{:,}'.format(port_worth_c))
    print("  median:       $" + '{:,}'.format(port_worth_m))
    print("  median no fee $" + '{:,}'.format(port_worth_m_no_fee))
    print("  aggressive:   $" + '{:,}'.format(port_worth_a))

    
    """
        Show how much these projections grew over the time period
        in terms of percentages
    """
    print("\n")
    print("Growth percentages")
    print("------------------")

    median_percentage = ((port_worth_m - p)/p)*100
    median_percentage_no_fee = ((port_worth_m_no_fee - p)/p)*100
    
    print("change median:        %.2f" % median_percentage)
    print("change median no fee  %.2f" % median_percentage_no_fee)
    
    
    """
        display the actual values per year
    """
    print("\n")
    print("Series values:")
    print_series("middle road", series_m)
    print("")
    print_series("middle road no fee", series_m_no_fee)

    
    """
        setup and plot the graph
    """
    print("\n")
    g_c = pd.Series(series_c)
    g_m = pd.Series(series_m)
    g_m_no_fee = pd.Series(series_m_no_fee)
    g_a = pd.Series(series_a)
    
    plt.rcParams['figure.figsize'] = [15, 9]
    plt.title("Plot of all the projections. One data point per year")
    plt.xlabel("years")
    plt.ylabel("value in $")
    g_c.plot(label='conservative')
    g_m.plot(label='middle road')
    g_m_no_fee.plot(label='middle road no fee')
    g_a.plot(label='agressive')

    # control the size of the ploy
    plt.legend() 
    
    return port_worth_m_no_fee


"""
    Run the actual program
"""


fv_itr = run_simulation_iterative(
                                  p=100, 
                                  r=0.1, 
                                  years=5, 
                                  expense_ratio=0.003, 
                                  annual_contribution=0)



Value of the portfolios after 5 years:
------------------------------------
  conservative: $160.80956840242277
  median:       $158.64968617141653
  median no fee $161.051
  aggressive:   $201.13571875


Growth percentages
------------------
change median:        58.65
change median no fee  61.05


Series values:
Portfolio: middle road
year | portfolio val
-----+--------------
0    | 100
1    | 109.67
2    | 120.275089
3    | 131.9056901063
4    | 144.66097033957922
5    | 158.64968617141653

Portfolio: middle road no fee
year | portfolio val
-----+--------------
0    | 100
1    | 110.0
2    | 121.0
3    | 133.1
4    | 146.41
5    | 161.051




NameError: name 'pd' is not defined

## Contributions
Contributions to this notebook are welcomed. The repo can be found [here](https://github.com/fat-fire/tools)